In [12]:
import pandas as pd
data= pd.read_csv("breast-cancer.csv")
data.head()

,Class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiate
0,no-recurrence-events,30-39,premeno,30-34,0-2,no,3,left,left_low,no
1,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,right,right_up,no
2,no-recurrence-events,40-49,premeno,20-24,0-2,no,2,left,left_low,no
3,no-recurrence-events,60-69,ge40,15-19,0-2,no,2,right,left_up,no
4,no-recurrence-events,40-49,premeno,0-4,0-2,no,2,right,right_low,no


In [13]:
from sklearn import preprocessing 
label_encoder = preprocessing.LabelEncoder() 
data['Class']= label_encoder.fit_transform(data['Class']) 
  
data['Class'].unique() 

array([0, 1])

In [14]:
data['age']= label_encoder.fit_transform(data['age'])   
data['age'].unique() 

array([1, 2, 4, 3, 5, 0])

In [15]:
data['menopause']= label_encoder.fit_transform(data['menopause'])
data['menopause'].unique()

array([2, 0, 1])

In [16]:
data['tumor-size']= label_encoder.fit_transform(data['tumor-size'])
data['inv-nodes']= label_encoder.fit_transform(data['inv-nodes'])
data['node-caps']= label_encoder.fit_transform(data['node-caps'])
data['breast']= label_encoder.fit_transform(data['breast'])
data['breast-quad']= label_encoder.fit_transform(data['breast-quad'])
data['irradiate'] = label_encoder.fit_transform(data['irradiate'])

In [17]:
data['irradiate'].value_counts()

0    218
1     68
Name: irradiate, dtype: int64

In [18]:
data.head()

,Class,age,menopause,tumor-size,inv-nodes,node-caps,deg-malig,breast,breast-quad,irradiate
0,0,1,2,6,0,1,3,0,2,0
1,0,2,2,4,0,1,2,1,5,0
2,0,2,2,4,0,1,2,0,2,0
3,0,4,0,3,0,1,2,1,3,0
4,0,2,2,0,0,1,2,1,4,0


In [19]:
# separate dataset into train and test
import pandas as pd

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler

# to correctly set up the cross-validation
from imblearn.pipeline import make_pipeline
from imblearn.under_sampling import NeighbourhoodCleaningRule
from imblearn.over_sampling import ADASYN
X_train, X_test, y_train, y_test = train_test_split(
    data.drop('Class', axis=1) , 
    data['Class'], # just the target
    test_size=0.3,
    random_state=0)

X_train.shape, X_test.shape

((200, 9), (86, 9))

In [20]:
# adasyn

adasyn = ADASYN(
        sampling_strategy='auto',  # samples only the minority class
        random_state=0,  # for reproducibility
        n_neighbors=5,
        n_jobs=4,
)

In [21]:
###################
## IMPORTANT  
##################=

# The sampling strategy needs to be set to all, or with
# a specific dictionary, because after ADASYN, our
# previous minority class is no longer minority!!

ncr = NeighbourhoodCleaningRule(
    sampling_strategy='all',# undersamples all classes
    n_neighbors=3, 
    kind_sel='mode', 
    threshold_cleaning=0.1, # the threshold to evaluate a class for cleaning (used only for clearning step)
) 

In [22]:
model = make_pipeline(
    MinMaxScaler(),
    adasyn,
    ncr, 
    RandomForestClassifier(
            n_estimators=100, random_state=39, max_depth=3, n_jobs=4
        ),
)

In [23]:
model.fit(X_train, y_train)

C:\Users\Vamshi\AppData\Local\Programs\Python\Python310\lib\site-packages\imblearn\under_sampling\_prototype_selection\_edited_nearest_neighbours.py:158: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will become False, the `axis` over which the statistic is taken will be eliminated, and the value None will no longer be accepted. Set `keepdims` to True or False to avoid this warning.
  nnhood_label, _ = mode(nnhood_label, axis=1)
C:\Users\Vamshi\AppData\Local\Programs\Python\Python310\lib\site-packages\imblearn\under_sampling\_prototype_selection\_edited_nearest_neighbours.py:158: FutureWarning: Unlike other reduction functions (e.g. `skew`, `kurtosis`), the default behavior of `mode` typically preserves the axis it acts along. In SciPy 1.11.0, this behavior will change: the default value of `keepdims` will be

Pipeline(steps=[('minmaxscaler', MinMaxScaler()),
                ('adasyn', ADASYN(n_jobs=4, random_state=0)),
                ('neighbourhoodcleaningrule',
                 NeighbourhoodCleaningRule(kind_sel='mode',
                                           sampling_strategy='all',
                                           threshold_cleaning=0.1)),
                ('randomforestclassifier',
                 RandomForestClassifier(max_depth=3, n_jobs=4,
                                        random_state=39))])

In [24]:
pred_train = model.predict_proba(X_train)[:,1]
pred_test = model.predict_proba(X_test)[:,1]

In [25]:
print('Train roc-auc: {}'.format(roc_auc_score(y_train, pred_train)))


Train roc-auc: 0.7910324708269914


In [26]:
print('Test roc-auc: {}'.format(roc_auc_score(y_test, pred_test)))

Test roc-auc: 0.7090909090909091
